# Break Downtrend Screener of S&P 500

Pull list of S&P 500 ticker symbols, and tried to detect a break in a stock price downtrend in the past 20 days.  Confirms against Relative Strength Index (RSI) and Moving average convergence/divergence (MACD).

https://www.investopedia.com/terms/r/rsi.asp

https://www.investopedia.com/terms/m/macd.asp

https://www.investopedia.com/terms/r/rsi.asp#toc-the-difference-between-rsi-and-macd

In [ ]:
pip install -r requirements.txt

In [2]:
from logging import exception

import yfinance as yf
import pandas as pd
import numpy as np
import requests
import time
from ta.momentum import RSIIndicator
from ta.trend import MACD

# --- 1. Get list of S&P 500 tickers from Wikipedia ---
def get_sp500_tickers():
    # url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    # response = requests.get(url)
    # soup = BeautifulSoup(response.text, 'lxml')
    # table = soup.find('table', {'id': 'constituents'})
    # tickers = [row.find_all('td')[0].text.strip() for row in table.find_all('tr')[1:]]
    # return tickers

    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    tables = pd.read_html(response.text)
    df = tables[0]
    tickers = df['Symbol'].tolist()

    # Replace '.' with '-' to match yfinance format (e.g., BRK.B → BRK-B)
    tickers = [ticker.replace('.', '-') for ticker in tickers]

    # Remove BRK-B and BF-B
    # tickers = [ticker for ticker in tickers if ticker not in ['BRK-B', 'BF-B']]

    return tickers


# --- 2. Check for downtrend break in last 20 days ---
def has_broken_downtrend_with_indicators(df):
    # Flatten the MultiIndex columns for easier access
    df.columns = df.columns.get_level_values(0)  # Get the first level of the MultiIndex

    # Check if 'Close' column exists
    if 'Close' not in df.columns:
        print("Error: 'Close' column not found!")
        return False, None  # Return None for the days ago value if 'Close' column is missing

    # Drop NaN values from 'Close' column only
    df = df.copy().dropna(subset=['Close'])

    # If df is empty after dropping NaNs, return False
    if df.empty:
        return False, None

    # Now, ensure prices is a 1D array
    prices = df['Close'].values.flatten()  # Ensure it's a 1D array

    # Ensure we have enough data
    if len(prices) < 30:
        return False, None

    # --- Downtrend check ---
    x = np.arange(len(prices))
    coef = np.polyfit(x, prices, 1)

    if coef[0] >= 0:
        return False, None  # Not a downtrend

    last_price = prices[-1]
    trend_today = coef[0] * (len(prices) - 1) + coef[1]
    broke_trend = last_price > trend_today

    if not broke_trend:
        return False, None

    # --- Track the day when downtrend breaks ---
    # Find where the break occurred (last price crosses the trend line)
    for i in range(len(prices) - 2, 0, -1):  # Check from the second-to-last back to the first
        trend_value = coef[0] * i + coef[1]
        if prices[i] <= trend_value and prices[i+1] > trend_value:
            break_day = df.index[i+1]  # The day the break occurred
            break

    # Calculate the number of days ago the break occurred
    days_ago = (df.index[-1] - break_day).days

    # --- Indicators ---
    rsi = RSIIndicator(close=df['Close'], window=14)
    df['RSI'] = rsi.rsi()

    macd = MACD(close=df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_signal'] = macd.macd_signal()

    # Drop NaN values from indicators
    df = df.dropna(subset=['RSI', 'MACD', 'MACD_signal'])

    if df.empty:
        return False, None

    latest = df.iloc[-1]
    rsi_pass = float(latest['RSI']) > 50
    macd_pass = float(latest['MACD']) > float(latest['MACD_signal'])

    return rsi_pass and macd_pass, days_ago



# --- 3. Loop through tickers and find breakouts ---
def find_confirmed_breakouts(tickers, verbose=False):
    confirmed = []
    for i, ticker in enumerate(tickers):
        try:
            df = yf.download(ticker, period='3mo', interval='1d', progress=False)

            # Check if df is valid and contains 'Close'
            if df is None or df.empty or 'Close' not in df.columns:
                print(f"⚠️ Skipping {ticker}: No 'Close' column or empty data")
                continue

            result, days_ago = has_broken_downtrend_with_indicators(df)

            if result:
               confirmed.append(ticker)
               if verbose:
                   print(f"{ticker} ✅ Downtrend Break {days_ago} days ago.")
        except Exception as e:
            print(f"❌ Error with {ticker}: {e}")
        time.sleep(1)
    return confirmed

# --- Run everything ---
if __name__ == "__main__":
    print("Fetching S&P 500 tickers...")
    tickers = get_sp500_tickers()
    print(f"Checking {len(tickers)} tickers for confirmed downtrend breaks...")
    confirmed_stocks = find_confirmed_breakouts(tickers, verbose=True)

    print("\n📈 Stocks with confirmed trend reversal (Downtrend Break + RSI > 50 + MACD > Signal):")
    for stock in confirmed_stocks:
        print(stock)



AXON ✅ Downtrend Break 1 days ago.
CNC ✅ Downtrend Break 15 days ago.
CMG ✅ Downtrend Break 1 days ago.
CTAS ✅ Downtrend Break 1 days ago.
CPRT ✅ Downtrend Break 1 days ago.
COST ✅ Downtrend Break 1 days ago.
CRWD ✅ Downtrend Break 1 days ago.
DVA ✅ Downtrend Break 17 days ago.
DLTR ✅ Downtrend Break 3 days ago.
HUM ✅ Downtrend Break 2 days ago.
LDOS ✅ Downtrend Break 17 days ago.
LYV ✅ Downtrend Break 1 days ago.
LMT ✅ Downtrend Break 1 days ago.
PWR ✅ Downtrend Break 3 days ago.
ROST ✅ Downtrend Break 14 days ago.
TJX ✅ Downtrend Break 1 days ago.
UNH ✅ Downtrend Break 20 days ago.
WMT ✅ Downtrend Break 1 days ago.

📈 Stocks with confirmed trend reversal (Downtrend Break + RSI > 50 + MACD > Signal):
AXON
CNC
CMG
CTAS
CPRT
COST
CRWD
DVA
DLTR
HUM
LDOS
LYV
LMT
PWR
ROST
TJX
UNH
WMT
